# Case Study: UK used BMW car price analysis and prediction

You are provided with a dataset containing information about used BMW cars, including their specifications and selling prices. The dataset contains information of price, transmission, mileage, fuel type, road tax, miles per gallon (mpg), and engine size. It is cool to have some insignts for the data by applying your learnt visualisation and modelling techniques, and see if we could determine what is the ideal price to sell your used BMW car (assuming you have one...) based on the known information. 

This notebook provides you a skeleton of analysis and modelling, but you should decide what information should be digged out and how the identified information could help you to setup a regression model to make an accuracy prediction for the selling price of your BMW car! At the end of this case study, you will be given another test set and you may find out how well your built model would perform. This is important, becuase that will determine if you could sell your car at a fair price!!

## 0. Import libraries

What Python libraries you will need? Import them for later usage.

## 1. Load and check the dataset

#### Hint - Try to answer the following questions:

1. How to load the dataset using pandas?
2. What columns should be features? What column should be predicting target?
3. How can you have a basic understanding for the given dataset?

## 2. preprocess the dataset

#### Hint - Try to answer the following questions:

1. Are there outliers missing values? 
2. How will you identify them and clean the dataset?

## 3. Data Visualistaion

#### Hint - Try to answer the following questions:

1. What visualisation tools you want to use?
2. How can these visualisation help you to understand the data?
3. How can your findings inspire the later modelling?

## 4. Modelling

#### Hint - Try to answer the following questions:

1. Any variable encoding needed? (e.g., dummy variable)
2. Which is the regression model you want to choose?
3. How to tune the hyperparameters? See [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
4. How well does your model could perform?